In [1]:
#!pip install polars numpy torch pandas seaborn

In [2]:
import polars as pl
import pandas as pd
import os
import sys
import random
import pickle
import torch
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
# set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
DATA_PATH = ""
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
import sys
obj = pickle.load(open(os.path.join(DATA_PATH, 'allevents_by_episode_48_charts'), 'rb'))
patients = obj.select(pl.col('subject_id')).to_series().to_list()
seqs = obj.select(pl.col('itemidx')).to_series().to_list()
mortality = obj.select(pl.col('mortality_tf')).to_series().to_list()

In [4]:
itemidx = set([each_item for events in seqs for items in events for each_item in items])
len(itemidx)

40119

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Custom dataset
from torch.utils.data import Dataset, DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, seqs, mortality):
        self.seqs = seqs
        self.labels = mortality
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, index):
        return self.seqs[index], self.labels[index]
dataset = CustomDataset(seqs, mortality)
assert len(dataset) == len(obj) #TODO write test separately 


In [7]:
# Collate function
def collate_fn(data):
    """
    Input data: a tuple of seqs, and label
    Output: x (num_patients, num_events, num_itemids) with *real* items in the events
            rev_x (num_patients, num_events, num_itemids) with *reverse real* items in the events
            masks  (num_patients, num_events, num_itemids) whether a *real* itemid in the events is present
            rev_masks (num_patients, num_events, num_itemids) whether a *reverse real* itemid in the events is present
    """
    seqs, labels = zip(*data)
    num_patients = len(seqs)
    max_num_events = max([len(event) for event in seqs])
    max_num_items = max([len(itemid) for event in seqs for itemid in event])
    tensor_shape = (num_patients, max_num_events, max_num_items)
    x =        torch.zeros(tensor_shape, dtype=torch.long)
    rev_x =    torch.zeros(tensor_shape, dtype=torch.long)
    masks =    torch.zeros(tensor_shape, dtype=torch.bool)
    rev_masks = torch.zeros(tensor_shape, dtype=torch.bool) 
    y =        torch.tensor(labels, dtype=torch.long)
    

    for i_patient, events in enumerate(seqs):
        for i_event, item in enumerate(events):
            padded_item = torch.concat([torch.tensor(item),
                                        torch.zeros(max_num_items - len(item))]).long()
            x[i_patient, i_event, :] = padded_item
            masks[i_patient, i_event, :] = torch.where(padded_item!=0,1,0)  
    for i_patient, events in enumerate(seqs):
        idx_all_real_events = torch.sum(x[i_patient, :, :], dim=(1))!= 0
        idx_padded_events =torch.sum(x[i_patient, :, :], dim=(1))== 0
        fliped = torch.flip(x[i_patient, idx_all_real_events, :].unsqueeze(1), (0,)).squeeze(1)
        rev_x[i_patient, :, :] = torch.concat((fliped, x[i_patient, idx_padded_events, :] ))
        rev_masks[i_patient, :, :] = torch.where(rev_x[i_patient, :, :] != 0, True, False)
    return x, masks, rev_x, rev_masks, y

    

from torch.utils.data import DataLoader
loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn, pin_memory=True)
loader_iter = iter(loader)
x, masks, rev_x, rev_masks, y = next(loader_iter)
#assert x.shape == masks.shape == (10, 8, 313)
assert y.shape == (10,)

In [8]:
train, val, test = int(len(dataset)*0.8), int(len(dataset)*0.1), len(dataset) - int(len(dataset)*0.8) -  int(len(dataset)*0.1)
lengths = [train, val, test]
train_dataset, val_dataset, test_dataset = random_split(dataset=dataset, lengths=lengths)
from pytorch_metric_learning import samplers

def load_data(train_dataset, val_dataset, test_dataset, collate_fn):
    batch_size = 64
    seqs, labels = zip(*train_dataset)
    
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, \
                                collate_fn=collate_fn, \
                                num_workers=4, shuffle=True)
    
    val_loader = DataLoader(train_dataset, batch_size=batch_size, \
                                collate_fn=collate_fn, \
                                num_workers=4, shuffle=False)
    test_loader = DataLoader(train_dataset, batch_size=batch_size, \
                                collate_fn=collate_fn, \
                                num_workers=4, shuffle=False)
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset, collate_fn)
print("Length of train dataset:", len(train_dataset))
print("Length of train with mortality:", np.sum([element[-1] for element in train_dataset]))
print("Length of train with survive patients:", (len(train_dataset) - np.sum([element[-1] for element in train_dataset])))
print("Length of val dataset:", len(val_dataset))
print("Length of val with mortality:", np.sum([element[-1] for element in val_dataset]))
print("Length of val with survive patients:", (len(train_dataset) - np.sum([element[-1] for element in val_dataset])))
print("Length of test dataset:", len(test_dataset))
print("Length of test with mortality:", np.sum([element[-1] for element in test_dataset]))
print("Length of test with survive patients:", (len(test_dataset) - np.sum([element[-1] for element in test_dataset])))

Length of train dataset: 27032
Length of train with mortality: 3542
Length of train with survive patients: 23490
Length of val dataset: 3379
Length of val with mortality: 458
Length of val with survive patients: 26574
Length of test dataset: 3379
Length of test with mortality: 471
Length of test with survive patients: 2908


In [9]:
def custom_for_loop(iterable):
    iterator = iter(iterable)
    done_looping = False
    count = 0
    while not done_looping:
        try:
            item = next(iterator)
        except StopIteration:
            done_looping = True
        else:
            x, masks, rev_x, rev_masks, y = item
            count += y.sum().item()
    print(count)

custom_for_loop(train_loader)

3542


In [10]:
def sum_embeddings_with_masks(x, masks):
    """
    Input:  x               (batch_size, num_events, num_itemids, embedding_dims)
            mask            (batch_size, num_events, num_itemids)
    Output: sum_embeddings  (batch_size, num_events, embedding_dims)
    The return output compress the num_itemids into embedding vectors
    """
    masks = masks.unsqueeze(-1).expand(x.shape[0], x.shape[1], x.shape[2], x.shape[3])
    return torch.sum(masks * x, 2)
def get_last_event(hidden_states, masks):
    """
    hidden_states: (batch_size, #item, embedding_dim)
    masks:         (batch_size, #item, embedding_dim)
    return last_hidden_state: (batch_size, embedding_dim)
    """
    idx_last_event = torch.argmin(torch.sum(masks, 2), 1)
    return hidden_states[torch.arange(hidden_states.shape[0]), torch.where(idx_last_event - 1 < 0, max(idx_last_event), idx_last_event - 1),:]

In [11]:
ts = torch.tensor([[
    [-0.8201, 0.3956, 0.8989, -1.3884, -0.1670, 0.2851, -0.6411],
    [-0.8937, 0.9265, -0.5355, -1.1597, -0.4602, 0.7085, 1.0128],
    [ 0.2304, 1.0902, -1.5827, -0.3246, 1.9264, -0.3300, 0.1984]],
   
[[ 0.7821, 1.0391, -0.7245, -0.2093, -0.2153, -1.8157, -0.3452],
    [-2.0615, 0.6741, -1.3233, -1.3598, -0.0835, -0.0235, 0.1744],
    [ 2.2983, 0.9571, -0.6619, -0.8285, -0.6057, -1.4013, 1.2973]],

   [[ 1.6409, -1.0567, -0.2616, -0.2501, 0.5011, 0.2600, -0.1782],
    [    -0.2595, -0.0145, -0.3839, -2.9662, -1.0606, -0.3090, 0.9343],
    [ 1.6243, 0.0016, -0.4375, -2.1085, 1.1450, -0.3822, -0.3553]],

   [[ 0.7542, 0.1332, 0.1825, -0.5146, 0.8005, -0.1259, -0.9578],
    [ 1.7518, 0.9796, 0.4105, 1.7675, -0.0832, 0.5087, -0.8253],
    [ 0.1633, 0.5013, 1.4206, 1.1542, -1.5366, -0.5577, -0.4383]]])
masks = torch.tensor([[[ True,  True,  True,  True, False],
         [ True,  True,  True, False, False],
         [False, False, False, False, False]],

        [[ True, False, False, False, False],
         [ True,  True,  True,  True, False],
         [ True,  True, False, False, False]],

        [[ True,  True, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[ True,  True,  True, False, False],
         [ True,  True,  True, False, False],
         [False, False, False, False, False]]])
assert torch.sum(get_last_event(ts, masks) == torch.tensor([
    [-0.8937, 0.9265, -0.5355, -1.1597, -0.4602, 0.7085, 1.0128],
    [ 2.2983, 0.9571, -0.6619, -0.8285, -0.6057, -1.4013, 1.2973],
    [1.6409, -1.0567, -0.2616, -0.2501, 0.5011, 0.2600, -0.1782],
    [ 1.7518, 0.9796, 0.4105, 1.7675, -0.0832, 0.5087, -0.8253],
    ])) == 7*4

In [12]:
# Naive RNN hehe :P
class NaiveRNN(nn.Module):
    def __init__(self, num_items):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=128)
        self.rnn = nn.GRU(128, hidden_size=128, batch_first=True)
        self.rev_rnn = nn.GRU(128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(in_features=128*2, out_features=1)
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x, masks, rev_x, rev_masks):
        batch_size = x.shape[0]
        # Forward pass with x and masks
        x = self.dropout(self.embedding(x))
        x = sum_embeddings_with_masks(x, masks)
        output, _ = self.rnn(x)
        real_event_h = get_last_event(output, masks)

        # Forward pass with rev_x and rev_masks
        rev_x = self.dropout(self.embedding(rev_x))
        rev_x = sum_embeddings_with_masks(rev_x, rev_masks)
        output_rev, _ = self.rnn(rev_x)
        real_event_h_rev = get_last_event(output_rev, rev_masks)

        # Concat both hidden states
        logits = self.dropout(self.fc(torch.cat([real_event_h, real_event_h_rev], 1)))
        probs = self.sigmoid(logits)
        return probs.view(batch_size)
naive_rnn = NaiveRNN(num_items=len(itemidx))
naive_rnn

NaiveRNN(
  (embedding): Embedding(40119, 128)
  (rnn): GRU(128, 128, batch_first=True)
  (rev_rnn): GRU(128, 128, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)

In [13]:
class CustomLoss(nn.Module):
    def __init__(self, num_majority, num_minority, K=0.01):
        super(CustomLoss, self).__init__()
        
        self.num_majority = num_majority
        self.num_minority = num_minority
        self.K = K
    def forward(self, y_hat, y):
        #y = torch.to('cpu').LongTensor(y)
        delta_maj, delta_min = self.K / self.num_majority**(1/4), self.K / self.num_minority**(1/4)
        zj = (y_hat > 0.5).int()
        masks = torch.where(zj != y)
        zc = y_hat
        sigma_zc_maj = torch.exp(y_hat - delta_maj) / \
                            (torch.exp(y_hat - delta_maj) + torch.sum(torch.exp(zj[masks])))

        sigma_zc_min = torch.exp(y_hat - delta_min) / \
                            (torch.exp(y_hat - delta_min) + torch.sum(torch.exp(zj[masks])))

        loss = (- torch.log(sigma_zc_maj) - torch.log(sigma_zc_min)).mean()
        return loss
minority = np.sum([element[-1] for element in train_dataset]) / len(train_dataset)
majority = 1 - minority
(majority, minority)


(0.8689701094998521, 0.13102989050014796)

In [14]:
criterion = CustomLoss(num_majority=majority, num_minority=minority)
optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=1e-3, weight_decay=1e-2)
from tqdm import tqdm
from sklearn.metrics import *
def train(model, train_loader, val_loader, n_epochs):
    for epoch in tqdm(range(n_epochs)):
        model.to(device).train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            x, masks, rev_x, rev_masks, y = x.to(device), masks.to(device), rev_x.to(device), rev_masks.to(device), y.to(device)
            y_hat = model(x, masks, rev_x, rev_masks).view(y.shape[0])
            loss = criterion(y_hat, y.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        acc, auc, precision, recall, f1score = eval_model(model, val_loader)
        print('Epoch: {} \t Validation acc: {:.2f}, auc:{:.2f}, precision: {:.2f}, recall: {:.2f}, f1: {:.2f}' 
              .format(epoch+1, acc, auc, precision, recall, f1score))    
def classification_metrics(Y_score, Y_pred, Y_true):
    acc, auc, precision, recall, f1score = accuracy_score(Y_true, Y_pred), \
                                           roc_auc_score(Y_true, Y_score), \
                                           precision_score(Y_true, Y_pred), \
                                           recall_score(Y_true, Y_pred), \
                                           f1_score(Y_true, Y_pred)
    return acc, auc, precision, recall, f1score



#precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
def eval_model(model, val_loader):
    model.to(device).eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    for x, masks, rev_x, rev_masks, y in val_loader:
        x, masks, rev_x, rev_masks, y = x.to(device), masks.to(device), rev_x.to(device), rev_masks.to(device), y.to(device)
        y_true = y.long().detach().to('cpu')
        y_hat = model(x, masks, rev_x, rev_masks).view(y.shape[0])
        y_score = y_hat.detach().to('cpu')
        y_pred = (y_hat > 0.5).int().detach().to('cpu')
    acc, auc, precision, recall, f1score = classification_metrics(y_score, y_pred, y_true)
    return acc, auc, precision, recall, f1score


In [15]:
# number of epochs to train the model
n_epochs = 50
train(naive_rnn, train_loader, val_loader, n_epochs)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 	 Training Loss: 7.885139


  2%|▏         | 1/50 [00:34<28:11, 34.52s/it]

Epoch: 1 	 Validation acc: 0.21, auc:0.55, precision: 0.21, recall: 1.00, f1: 0.34
Epoch: 2 	 Training Loss: 7.877917


  4%|▍         | 2/50 [01:08<27:22, 34.22s/it]

...

Epoch: 48 	 Validation acc: 0.21, auc:0.60, precision: 0.21, recall: 1.00, f1: 0.34
Epoch: 49 	 Training Loss: 7.873395


 98%|█████████▊| 49/50 [27:08<00:32, 32.55s/it]

Epoch: 49 	 Validation acc: 0.21, auc:0.60, precision: 0.21, recall: 1.00, f1: 0.34
Epoch: 50 	 Training Loss: 7.873006


100%|██████████| 50/50 [27:22<00:00, 32.85s/it]

Epoch: 50 	 Validation acc: 0.21, auc:0.59, precision: 0.21, recall: 1.00, f1: 0.34


In [16]:
acc, auc, precision, recall, f1score = eval_model(naive_rnn, val_loader)
print(acc, auc, precision, recall, f1_score)

0.20833333333333334 0.5894736842105264 0.20833333333333334 1.0 <function f1_score at 0x7fcf58795550>


In [17]:
acc, auc, precision, recall, f1score = eval_model(naive_rnn, test_loader)
print(acc, auc, precision, recall, f1_score)

0.20833333333333334 0.5894736842105264 0.20833333333333334 1.0 <function f1_score at 0x7fcf58795550>
